In [1]:
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import re 
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/dval/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#open non-kp tweets
with open('non_kp_tweets.pickle', "rb") as f:
    tweets = pickle.load(f)

In [4]:
#identify tweets labeled as pashto and delete from dataset
twitter_pashto = tweets[tweets["tweet_lang"]=="ps"]
tweets = tweets[tweets["tweet_lang"]!="ps"]

In [6]:
#upload user created-pashto stopwords and search in tweets, deleting them from dataset
pashto_stopwords = pd.read_excel("new_pashto_stopwords.xlsx")
pashto_stopwords = pashto_stopwords["word"].to_list()

def exact_match(stopwords, string):
    for word in stopwords:
        if re.search(r'\b' + word + r'\b', string):
            return True
        else:
            return False
        
tweets["pashto"] = tweets.text.apply(lambda x: exact_match(pashto_stopwords, x))
pashto_from_stopwords = tweets[tweets["pashto"]==True]
tweets = tweets[tweets["pashto"]==False]

In [6]:
#Concat both sets of pashto language tweets and save
pashto_tweets = pd.concat([twitter_pashto, pashto_from_stopwords], ignore_index=True)

In [17]:
#Identify user IDs of people who tweeted in pashto and gather all of their tweets in all languages
pashto_users = pashto_tweets[["user_id"]]
pashto_users = pashto_users["user_id"].unique()
pashto_users = pd.DataFrame(pashto_users)
pashto_users.columns = ["user_id"]
pashto_tweets = tweets.merge(pashto_users, on = "user_id")

In [23]:
#Separate Pashto diaspora tweets by language and save
pashto_tweets = pashto_tweets.dropna()
urdu_pashto_tweets = pashto_tweets[pashto_tweets["tweet_lang"]=="ur"]
english_pashto_tweets = pashto_tweets[pashto_tweets["tweet_lang"]=="en"]

with open("non_kp_tweets_pashto.pickle", "wb") as f:
    pickle.dump(pashto_tweets, f)
    
with open("non_kp_tweets_non_pashto.pickle", "wb") as f:
    pickle.dump(tweets, f)
    
with open("non_kp_tweets_urdu_pashto.pickle", "wb") as f:
    pickle.dump(urdu_pashto_tweets, f)
    
with open("non_kp_tweets_english_pashto.pickle", "wb") as f:
    pickle.dump(english_pashto_tweets, f)